# Model

In [ ]:
# save the final model to file
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = mnist.load_data()
	# reshape dataset to have a single channel
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# fit model
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
	# save model
	model.save('final_model.h5')
  # evaluate model on test dataset
  
 
#entry point, run the test harness
#run_test_harness()

# interface

In [1]:
import os
from tkinter import *
import PIL
from PIL import ImageGrab



class main:
    def __init__(self, master):
        self.master = master
        self.res = ""
        self.pre = [None, None]
        self.bs = 8.5
        self.c = Canvas(self.master,bd=3,relief="ridge", width=300, height=282, bg='white')
        self.c.pack(side=LEFT)
        f1 = Frame(self.master, padx=5, pady=5)
        Label(f1,text="CNN with khaoula",fg="green",font=("",15,"bold")).pack(pady=10)
        Label(f1,text="MNSIT Dataset",fg="green",font=("",15)).pack()
        
        self.pr = Label(f1,text="Prediction: None",fg="blue",font=("",20,"bold"))
        self.pr.pack(pady=20)
        
        Button(f1,font=("",15),fg="white",bg="red", text="Clear ", command=self.clear).pack(side=BOTTOM)

        f1.pack(side=RIGHT,fill=Y)
        self.c.bind("<Button-1>", self.putPoint)
        self.c.bind("<ButtonRelease-1>",self.getResult)
        self.c.bind("<B1-Motion>", self.paint)

    def getResult(self,e):
        x = self.master.winfo_rootx() + self.c.winfo_x()
        y = self.master.winfo_rooty() + self.c.winfo_y()
        x1 = x + self.c.winfo_width()
        y1 = y + self.c.winfo_height()
        img = PIL.ImageGrab.grab()
        img = img.crop((x, y, x1, y1))
        img.save("dist.png")
        self.res = str(predict("dist.png"))
        self.pr['text'] = "Prediction: " + self.res

    def clear(self):
        self.c.delete('all')

    def putPoint(self, e):
        self.c.create_oval(e.x - self.bs, e.y - self.bs, e.x + self.bs, e.y + self.bs, outline='black', fill='black')
        self.pre = [e.x, e.y]

    def paint(self, e):
        self.c.create_line(self.pre[0], self.pre[1], e.x, e.y, width=self.bs * 2, fill='black', capstyle=ROUND,
                           smooth=TRUE)

        self.pre = [e.x, e.y]


if __name__ == "__main__":
    root = Tk()
    main(root)
    root.title('Digit')
    root.resizable(0, 0)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\programms\an\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\asus\AppData\Local\Temp\ipykernel_17656\4212354685.py", line 38, in getResult
    self.res = str(predict("dist.png"))
NameError: name 'predict' is not defined


In [1]:
import keras
from keras.models import load_model
from scipy import misc, spatial
from PIL import Image
import numpy as np
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array

model = load_model("final_model.h5")
                
def predict(InputImg):
    image = load_img(InputImg, grayscale=True, target_size=(28, 28))
    image = np.invert(image)
    image = img_to_array(image)
    image = image.reshape(1,28,28,1)

    return model.predict(image)[0].tolist().index(1)

In [4]:
import gradio as gr
import keras
from keras.models import load_model
from scipy import misc, spatial
from PIL import Image
import numpy as np
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
model = load_model("final_modell.h5")
def predict_image(img):
    
                
    img_3d=img.reshape(-1,28,28)
    im_resize=img_3d/255.0
    prediction=model.predict(im_resize)
    pred=np.argmax(prediction)
    return pred

iface = gr.Interface(predict_image, inputs="sketchpad", outputs="label")
iface.launch(debug='True')

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 29ms/step
Keyboard interruption in main thread... closing server.
